<div class="alert alert-success" role="alert">
  <h1 class="alert-heading">Working with the Chinese Calligraphy Styles dataset*</h1>
  <p><i>*By Kauvin Lucas, submitted in Kaggle[1] and Jovian[2]. More details in the references section.</i></p>

![header](https://i.imgur.com/XzJ2RvS.png)

Chinese calligraphy is a very unique visual art and an important manifestation of Chinese ancient culture which is popular with many people in the world [3]. As with any other artwork, Chinese calligraphy can take several years to master and have a high economic value for the holders. 

Although it's a form of art, there are traditional, strict rules that must be followed to make a legitimate calligraphy. Still, the style of each calligraphy is unique and anyone looking at it can tell its caligrapher.

In this notebook, we'll build and test a simple Convolutional Neural Network (CNN) model with 9 layers without batch normalization to identify the author's initials of each available font. Compared to other classification models, CNN performs well without feature design [4].

### 1 - Install and import the required libraries

In [ ]:
!pip install jovian --upgrade -q

In [ ]:
import os
import torch
import torchvision
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import jovian

### 2 - Load and transform the the dataset

In [ ]:
data_dir = '../input/chinese-calligraphy-styles-by-calligraphers/data/data'

In [ ]:
dataset = ImageFolder(data_dir+'/train', transform=ToTensor())

### 3 - Explore the dataset

#### 3.1 - How many classes are in the dataset?

In [ ]:
classes = os.listdir(data_dir + "/train")
print(os.listdir(data_dir))
print(classes)

There are a total of 20 classes in the dataset

#### 3.2 - How many examples are in each class?

In [ ]:
for class_ in classes:
    print("Class "+ class_ + ": " + str(len(os.listdir(data_dir + "/train/" + class_))))

All of the classes contain a total of 82022 images

#### 3.3 - Visualize 3 examples

In [ ]:
def show_example(img, label):
    print('Label: ', dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))

In [ ]:
# Example from class 1
img1, label1 = dataset[0]
print(img1.shape, label1)
print(img1)
show_example(*dataset[0])

In [ ]:
# Example from class 9
img2, label2 = dataset[40000]
print(img2.shape, label2)
print(img2)
show_example(*dataset[40000])

In [ ]:
# Example from class 12
img2, label2 = dataset[55000]
print(img2.shape, label2)
print(img2)
show_example(*dataset[55000])

### 4 - Split the dataset

In [ ]:
random_seed = 1234
torch.manual_seed(random_seed);

val_size = 8202 # 10% of the total size
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

### 5 - Load data in batches

In [ ]:
batch_size=128
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)

In [ ]:
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break

In [ ]:
show_batch(train_dl)

### 6 - Modelling

#### 6.1 - Test a simple model

In [ ]:
simple_model = nn.Sequential(
    nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(2, 2)
)

In [ ]:
for images, labels in train_dl:
    print('images.shape:', images.shape)
    out = simple_model(images)
    print('out.shape:', out.shape)
    break

If our simple model has been called without problems, we'll proceed to define our main CNN model

#### 6.2 - Define helper functions for training and validation
We will use the Cross Entropy loss function for calculating loss in the training and validation steps

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

#### 6.3 - Chain layers into a single network architecture

In [ ]:
class Cifar10CnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            nn.Linear(16384, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 20))
        
    def forward(self, xb):
        return self.network(xb)

In [ ]:
model = Cifar10CnnModel()
model

In [ ]:
for images, labels in train_dl:
    print('images.shape:', images.shape)
    out = model(images)
    print('out.shape:', out.shape)
    print('out[0]:', out[0])
    break

#### 6.4 - Build helper functions to move model and data to a CUDA device

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device);

### 7 - Training

#### 7.1 - Define functions to fit and evaluate the model

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

#### 7.2 - Move model to device

In [ ]:
model = to_device(Cifar10CnnModel(), device)

#### 7.3 - Evaluate the model by its initial parameters

In [ ]:
evaluate(model, val_dl)

#### 7.4 - Define and log hyperparameters

In [ ]:
# Define hyperparameters
num_epochs = 6 # Number of epochs, enough to prevent overfitting
opt_func = torch.optim.Adam # Implements Adam algorithm for optimization
lr = 0.001 # Learning rate

In [ ]:
# Log hyperparamenters to Jovian
jovian.reset()
jovian.log_hyperparams({
    'num_epochs': num_epochs,
    'opt_func': opt_func.__name__,
    'batch_size': batch_size,
    'lr': lr,
})

#### 7.5 - Fit model and log metrics

In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

In [ ]:
# log fitted model metrics to Jovian
jovian.log_metrics(train_loss=history[-1]['train_loss'], 
                   val_loss=history[-1]['val_loss'], 
                   val_acc=history[-1]['val_acc'])

#### 7.6 - Plot accuracy and loss history

In [ ]:
# Plot accuracy history
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');
plot_accuracies(history)

In [ ]:
# Plot loss history
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

In [ ]:
plot_losses(history)

### 8 - Testing

#### 8.1 - Transform test data

In [ ]:
test_dataset = ImageFolder(data_dir+'/test', transform=ToTensor())

#### 8.2 - Predict and compare labels (5 examples)

In [ ]:
def predict_image(img, model):
    # Convert to a batch of 1
    xb = to_device(img.unsqueeze(0), device)
    # Get predictions from model
    yb = model(xb)
    # Pick index with highest probability
    _, preds  = torch.max(yb, dim=1)
    # Retrieve the class label
    return dataset.classes[preds[0].item()]

In [ ]:
img, label = test_dataset[0]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model))

In [ ]:
img, label = test_dataset[1002]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model))

In [ ]:
img, label = test_dataset[6153]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model))

In [ ]:
img, label = test_dataset[12000]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model))

In [ ]:
img, label = test_dataset[14560]
plt.imshow(img.permute(1, 2, 0))
print('Label:', dataset.classes[label], ', Predicted:', predict_image(img, model))

#### 8.4 - Look for overall loss and accuracy on test data

In [ ]:
test_loader = DeviceDataLoader(DataLoader(test_dataset, batch_size*2), device)
result = evaluate(model, test_loader)
result

#### 8.5 - Save the model once it's done

In [ ]:
torch.save(model.state_dict(), 'cifar10-cnn.pth')

In [ ]:
model2 = to_device(Cifar10CnnModel(), device)

In [ ]:
model2.load_state_dict(torch.load('cifar10-cnn.pth'))

### 9 - Final commit to Jovian

In [ ]:
jovian.commit(project="calligraphy-style-classification")

### 10 - Conclusion and final remarks

We've defined our model with 9 layers, and after trying with different hyperparamenters, we arrived at a model that performed quite well and generated an accuracy of **96%** on the test data. No batch normalization was employed since this overfitted the model.

This notebook was intented to explore simple CNN models to predict the caligraphers initials behind each calligraphy font. Of course, a few things could be done to improve the model like applying augmentation schemes on the training data.

### 11 - References

[1] "Calligraphy Style Classification", by Kauvin Lucas in Kaggle: https://www.kaggle.com/kauvinlucas/calligraphy-style-classification.

[2] "Calligraphy Style Classification", by Kauvin Lucas in Jovian: https://jovian.ai/kauvinlucas/calligraphy-style-classification.

[3] Auto-Encoder Guided GAN for Chinese Calligraphy Synthesis, by Pengyuan Lyu, Xiang Bai, Cong Yao, Zhen Zhu, Tengteng Huang, Wenyu Liu1 in 2017 14th IAPR International Conference on Document Analysis and Recognition (ICDAR): https://doi.org/10.1109/ICDAR.2017.181

[4] Recognizing Chinese Calligraphy Styles: A Cage Fight, by Chen Yu-Sheng, Li Haihong, Su Guangjun in Stanford University:http://cs229.stanford.edu/proj2016/poster/ChenSuLi-Machine%20Learning%20for%20Different%20Calligraphy%20Style%20Recognition-poster.pdf